# Audio Reconstruction - auto encoders

## Architecture générale : CNN ?

Avant de passer à la régression de paramètres de synthé, on va d'abord entraîner un AE à reconstruire les (mel-spectrogrammes) des sons des presets du DX7.

Choix du AE :
* Encodeur/décodeur à CNN sur spectrogramme (FlowSynth, IRCAM 2019)
    * Encodeur (décodeur idem) : 5 couches de 64 canaux, kernel 7, stride 2, dilation 2
    * WAE, qui d'après (Esling et al. 2020) donne au final les meilleures régressions de paramètres de synthé ?
    * Divers VAE avec flow, qui d'après (Esling et al. 2020) donnent la meilleure reconstruction audio ? (avec metadata....)
        * (https://acids-ircam.github.io/flow_synthesizer/#models-details)
* WaveNet
    * 10 couches de kernel 4, stride 2, dilation 1. Nombre de canaux va de 128-512 à 1024 pour mix final
    * Temporal encoder à convolution sur raw audio
        * Comparer le *baseline CNN encoder* avec le *temporal encoder* sur spectrogrammes ? Même si Google Brain/DeepMind restent sur du CNN classique 
    * Décodeur WaveNet : beaucoup trop lourd
* WaveNet baseline (and temporal) :
![](https://pbs.twimg.com/media/C8spAMCXYAAX6ml.jpg)

---
---

## Questions en suspens :
* Est-ce c'est intéressant d'avoir un décodeur qui génère une image légèrement trop large ? (Pour limiter les effets de bords sur les dernières couches)
* Comment implémenter la régularisation MMD du WAE ?
    * Avec "Inverse Multiquadratics" kernel, cf. WAE 2018
* VAE avec normalizing flow : est-ce qu'on fait la régularisation au début ou à la fin du flow ?
    * Pour WAE : même problème...
* Intégrer le pitch qqpart va sans doute beaucoup aider
    * Pour comprendre tous les params, plusieurs notes//plusieurs pitch sont probablement nécessaires
* Problème général des synthés qui peuvent générer le même son de plusieurs manières différentes

## Dimension de l'espace latent z :
* FlowSynth (Esling et al. 2020) prend $d_z = d_v$ avec $v$ le vecteur des paramètres. En effet, ils ont voulu un flow inversible entre z et v
    * $+$ : traduction facile d'un preset en vecteur latent z
    * $+$ : de chaque côté de l'interpolation, on a *exactement* le preset d'origine...
    * $+$ : chaque réel $z_i$ peut se rapprocher d'une dimension perceptuelle de haut-niveau
    * $-$ : limite de taille de l'espace latent
    * $-$ : autant de "macro-contrôles" que de params de synthé (solution possible : interp miem...)
    * $-$ : MSE assez forte sur valeurs de paramètres (flow inversible pas si puissant qu'un NN)
* WaveNet AE (Engel et al. 2017) encode un z à 2 dimensions : 16 dimensions par timestep, 125 timesteps de 512 samples (32 ms à 16kHz)
    * Un latent z comme ça permet sans doute de bien représenter attaque/sustain/decay, LFO, etc ? Et donc d'obtenir une bonne reconstruction
   

## Loss

### Coût de reconstruction
* MSE Loss
* Spectral convergence ?

### Régularisation
* D_KL
* MMD

## Recherche pour publication début 2021, spécifique sur l'interpolation de paramètres

Se focaliser sur l'entraînement et la généralisation à tout synthé - pour interpolation de presets.

* *Structure de base : WAE avec enc/dec à CNN sur spectrogramme ?*
    * *(on pourrait aussi essayer le temporal encoder wavenet)*
* Extension du taf de Esling et al. 2019/2020 
    * Résultats très encourageants, mais il y a clairement *room for improvement*
        * Nb de params du synthé : limités
        * À tester sur d'autres synthés, comme le DX7 (synthèse + complexe mais pas d'effets intégrés)
    * Dans l'objectif de l'interp. de preset
        * **Étude de continuité sonore de l'interpolation ?**
        * Pour n'importe quel synth, avec peu de presets, zéro meta-tag
        * Pure unsupervised learning
        * **Étude applicative, en explorant différents synthés et différents spectrogrammes ?**
            * *Avantage de faire ça : peu importe que ça ne marche pas bien avec un synthé ou l'autre...*
    * **Tests de fonctions de Loss pondérées sur spectrogr**
        * Facteur de loss + important sur les basses fréquences (Mel-Loss)
        * Facteur de loss + important à proximité de l'attaque et du release
    * Leurs résultats comparaient déjà les 2 approches intéressantes
        * **Flow inversible z <-> v, qui est la solution idéale pour l'interpolation**
            * Mais FlowSynth sans Flow : 2-layer MLP 1024 n'est pas très profond (d'où : - expressif que Flow)
        * WAE avec MLP z -> v ??
            * MLP régression sauvage
            * MLP avec modélisation fine (sortie cat. vs sorties gaussiennes)
        * **Tester d'autres NN inversibles ?** Sans forcément modéliser toutes les maths...?
            * Le domaine des INN, c'est un peu le domaine des flows...
    * Autre expé avec le DX7, avec comme objectifs
        * Confirmer ou infirmer résultats, pour un autre synthé
            * En utilisant *tous* les paramètres du synthé, pas un subset choisi (max 64 sur 200+ dans Diva)
            * **Problème du pitch/vel qui influe --> entraîner un NN AVEC PITCH ET VEL DANS LE VAE ?? Il faudrait plusieurs spectrogrammes pour 1 seule estimation**
        * Essayer d'améliorer la modélisation de l'audio avec NNs plus gros
        * Tester **data augmentation** en modifiant légèrement certaines valeurs de paramètres avant de générer en live un spectrogramme
            * Essayer un synthé avec beaucoup moins de preset. En pratique, on a rarement 10000 ou 30000 presets (et qualité discutable) comme avec diva et/ou le DX7.
            * Problèmes en multi-processing (deadlock dans selector.py) avec RenderMan ? Pas sûr : PIP avait le même blocage. **Ré-essayer le rendu audio temps-réel**
        * Discuter des avantages/inconvénients du MLP z->v vs. flow z<->v
    * GAN - AutoEncoder ?
        * Voir s'il existe une structure pour ça, déjà...
        * Ça ferait un double Loss, mais peut-être meilleur que MSE
        
---
---

# Auto-encoder audio seul

## Visualisation des tailles de tenseur entrée/sortie

In [1]:
from data import dataset
import importlib
importlib.reload(dataset)

dexed_dataset = dataset.DexedDataset()

/home/gwendal/anaconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [4]:
import torch
import importlib

import torchinfo
from model import encoder
from model import decoder
import model.layer
from model import VAE
import config

def reload_all_modules():
    importlib.reload(encoder)
    importlib.reload(decoder)
    importlib.reload(model.layer)
    importlib.reload(model.VAE)
    importlib.reload(config)
reload_all_modules()

# - - - Copié depuis train.py - - -
encoder_model = encoder.SpectrogramEncoder(config.model.encoder_architecture, config.model.dim_z,
                                           config.model.spectrogram_size)
# - - - fin de copie - - -
print("Encoder Architecture = {}".format(config.model.encoder_architecture))
_ = torchinfo.summary(encoder_model, input_size=config.model.input_tensor_size, depth=5, device='cpu')

Encoder Architecture = speccnn8l1
Layer (type:depth-idx)                   Output Shape              Param #
├─SpectrogramCNN: 1-1                    [192, 1024, 3, 4]         --
|    └─Sequential: 2-1                   [192, 1024, 3, 4]         --
|    |    └─Conv2D: 3-1                  [192, 8, 129, 174]        --
|    |    |    └─Conv2d: 4-1             [192, 8, 129, 174]        208
|    |    |    └─LeakyReLU: 4-2          [192, 8, 129, 174]        --
|    |    |    └─BatchNorm2d: 4-3        [192, 8, 129, 174]        16
|    |    └─Conv2D: 3-2                  [192, 16, 65, 88]         --
|    |    |    └─Conv2d: 4-4             [192, 16, 65, 88]         2,064
|    |    |    └─LeakyReLU: 4-5          [192, 16, 65, 88]         --
|    |    |    └─BatchNorm2d: 4-6        [192, 16, 65, 88]         32
|    |    └─Conv2D: 3-3                  [192, 32, 33, 45]         --
|    |    |    └─Conv2d: 4-7             [192, 32, 33, 45]         8,224
|    |    |    └─LeakyReLU: 4-8          [19

In [14]:
# Summary on the DECODER alone
reload_all_modules()
# - - - Copié depuis train.py - - -
decoder_model = decoder.SpectrogramDecoder(config.model.encoder_architecture, config.model.dim_z,
                                           config.model.spectrogram_size)
# - - - fin de copie - - -
print("Decoder Architecture = {}".format(config.model.encoder_architecture))  # Same as encoder
_ = torchinfo.summary(decoder_model, input_size=(1, config.model.dim_z), depth=5, device='cpu')

Decoder Architecture = speccnn8l1
Layer (type:depth-idx)                        Output Shape              Param #
├─Linear: 1-1                                 [1, 12288]                3,158,016
├─SpectrogramCNN: 1-2                         [1, 1, 257, 347]          --
|    └─Sequential: 2-1                        [1, 1, 257, 347]          --
|    |    └─TConv2D: 3-1                      [1, 512, 3, 4]            --
|    |    |    └─ConvTranspose2d: 4-1         [1, 512, 3, 4]            524,800
|    |    |    └─LeakyReLU: 4-2               [1, 512, 3, 4]            --
|    |    |    └─BatchNorm2d: 4-3             [1, 512, 3, 4]            1,024
|    |    └─TConv2D: 3-2                      [1, 256, 5, 7]            --
|    |    |    └─ConvTranspose2d: 4-4         [1, 256, 5, 7]            2,097,408
|    |    |    └─LeakyReLU: 4-5               [1, 256, 5, 7]            --
|    |    |    └─BatchNorm2d: 4-6             [1, 256, 5, 7]            512
|    |    └─TConv2D: 3-3              

In [34]:
# Full-model summary
reload_all_modules()
# - - - Copié depuis train.py - - -
ae_model = VAE.BasicVAE(encoder_model, config.model.dim_z, decoder_model)
# - - - fin de copie - - -
_ = torchinfo.summary(ae_model, input_size=config.model.input_tensor_size, depth=5, device='cpu')

Layer (type:depth-idx)                             Output Shape              Param #
├─SpectrogramEncoder: 1-1                          [32, 2, 256]              --
|    └─SpectrogramCNN: 2-1                         [32, 1024, 5, 5]          --
|    |    └─Sequential: 3-1                        [32, 1024, 5, 5]          --
|    |    |    └─Conv2D: 4-1                       [32, 8, 257, 217]         --
|    |    |    |    └─Conv2d: 5-1                  [32, 8, 257, 217]         208
|    |    |    |    └─LeakyReLU: 5-2               [32, 8, 257, 217]         --
|    |    |    |    └─BatchNorm2d: 5-3             [32, 8, 257, 217]         16
|    |    |    └─Conv2D: 4-2                       [32, 16, 129, 109]        --
|    |    |    |    └─Conv2d: 5-4                  [32, 16, 129, 109]        2,064
|    |    |    |    └─LeakyReLU: 5-5               [32, 16, 129, 109]        --
|    |    |    |    └─BatchNorm2d: 5-6             [32, 16, 129, 109]        32
|    |    |    └─Conv2D: 4-3   